In [94]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pyspark import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [95]:
# Creating the spark session
#.config("spark.driver.extraClassPath", Jars) - Optional

Jars = "C:/Spark/spark-3.3.0-bin-hadoop3/jars/mssql-jdbc-10.2.1.jre8.jar; C:/Spark/spark-3.3.0-bin-hadoop3/jars/sqljdbc42.jar; C:/Spark/spark-3.3.0-bin-hadoop3/jars/sqljdbc_auth.dll"

spark = SparkSession.builder\
.appName("application one")\
.config("spark.master", "local[*]")\
.getOrCreate()

In [96]:
# Connection string to SQL SERVER

server_name = "jdbc:sqlserver://LOHITHGK"
database_name = "AdWorks"
url = server_name + ";" + "databaseName=" + database_name + ";" + "encrypt=true"+ ";" + "trustServerCertificate=true" + ";" + "integratedSecurity=true"

In [97]:
# Read the data from SQL SERVER table into the dataframe. 
# .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\ -- Optional

person_add=spark.read\
.format('jdbc')\
.option('url', url)\
.option('dbtable', 'AdWorks.dbo.Employee_output')\
.load()

In [98]:
# create Temp view or Replace the existing view.
person_add.createOrReplaceTempView('Employee_output_temp')

In [99]:
# Query the temp View using Spark SQL.
HumanResources_Shift_f = spark.sql("""
select * 
from Employee_output_temp
""")

In [100]:
# Write the dataframe data to SQL db table. 
# If the table is not existed, then it would create the table and then perform write operation.
# .mode('append') - append the the data, .mode('overwrite') - overwrite the exisiting data.

HumanResources_Shift_f.write\
    .format('jdbc')\
    .mode('append')\
    .option("truncate","true")\
    .option('url', url)\
    .option('dbtable', 'AdWorks.dbo.Employee_output_test')\
    .save()